In [7]:
!pip3 install canvasapi --quiet
!pip3 install pyinstaller --quiet

In [8]:
import canvasapi

In [10]:
TOKEN = "6936~sfusmUWOeQdmhxPUkpd0emeG0laShQCJZkRG2R86DzJUf2gfvVOQwrxCz9TZp83f"  # pls don't hack me 
URL = "https://k12.instructure.com/"